# Downloading data with Google Earth Engine

In [ ]:
import numpy as np
import math
import scipy
import pandas as pd
import PIL
import gdal
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import sys, os
from pathlib import Path
import time
import random
import collections, functools, operator
import csv
from osgeo import gdal,osr
from gdalconst import *
import subprocess
from osgeo.gdalconst import GA_Update
import datetime

import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
def ee_get_intersections(collection1, collection2, n1, n2):
    dates1 = collection1.aggregate_array('system:time_start').getInfo()
    dates1 = [datetime.datetime.fromtimestamp(x // 1000).strftime('%Y-%m-%d') for x in dates1]
    dates2 = collection2.aggregate_array('system:time_start').getInfo()
    dates2 = [datetime.datetime.fromtimestamp(x // 1000).strftime('%Y-%m-%d') for x in dates2]
    datesIntersections = []
    print(dates1)
    print(dates2)
    for date in dates1:
        if date in dates2:
            datesIntersections.append(date)
    return datesIntersections


def image_type(img, type):
    types = {   "float":img.toFloat(), 
                "byte":img.toByte(), 
                "int":img.toInt(),
                "double":img.toDouble(),
                "long": img.toLong(),
                "short": img.toShort(),
                "int8": img.toInt8(),
                "int16": img.toInt16(),
                "int32": img.toInt32(),
                "int64": img.toInt64(),
                "uint8": img.toUint8(),
                "uint16": img.toUint16(),
                "uint32": img.toUint32()}
  
    return types[type]

def ee_imageToTask(img, scale, colID=None, crs=None, id=None):
    if id == None:
        try:
            id = img.id().getInfo()
        except:
            id = colID+'_image_'+str(i)

    img = image_type(img, "int32")
    
    return ee.batch.Export.image.toDrive(image=img,
                            description=id,
                            folder=folder,
                            fileNamePrefix=id,
                            scale=scale, # 30 meters per pixel, set this value according to satellite imagery documentation
                            maxPixels=1e13,
                            crs=crs) # set as high as possible




## Downloads Hyperion and Landsat day-matching scenes over an area of interest

In [ ]:
folder = 'GEECollection Projection 32611'


collection1 = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filter(ee.Filter.date('2014-01-01', '2014-12-31'))
                                                            .filter(ee.Filter.eq('WRS_PATH', 144))
                                                            .filter(ee.Filter.eq('WRS_ROW', 52)))
collection2 = (ee.ImageCollection('EO1/HYPERION').filter(ee.Filter.date('2001-05-01', '2017-03-13'))
                                                            .filter(ee.Filter.eq('WRS_PATH', 144))
                                                            .filter(ee.Filter.eq('WRS_ROW', 52)))


n1 = collection1.size().getInfo()
print(n1)
colList1 = collection1.toList(n1)
colID1 = collection1.getInfo()['id'].replace('/','_')
img = ee.Image(colList1.get(1))
print(datetime.datetime.fromtimestamp((img.date().getInfo()['value'] // 1000)).strftime('%Y-%m-%d'))
n2 = collection2.size().getInfo()
print(n2)
colList2 = collection2.toList(n2)
colID2 = collection2.getInfo()['id'].replace('/','_')


datesIntersections = ee_get_intersections(collection1, collection2, n1, n2)

print(datesIntersections)

tasks1 = []
for i in range(n1):
    img = ee.Image(colList1.get(i))
    date =  datetime.datetime.fromtimestamp((img.date().getInfo()['value']) // 1000).strftime('%Y-%m-%d')
    if date in datesIntersections:
        tasks1.append(ee_imageToTask(img, 30, colID1))

tasks2 = []
for i in range(n2):
    img = ee.Image(colList2.get(i))
    date =  datetime.datetime.fromtimestamp((img.date().getInfo()['value']) // 1000).strftime('%Y-%m-%d')
    if date in datesIntersections:
        tasks2.append(ee_imageToTask(img, 30, colID2))


print(tasks1)
print(tasks2)

## Downloads California area data (Hyperion and USDA NASS Crop Data Layer)

In [ ]:
AOI = ee.Geometry.Polygon([[[-120.36302942846854, 36.950654941997534],
                    [-120.36302942846854, 35.889823731552326],
                    [-119.74230188940604, 35.889823731552326],
                    [-119.74230188940604, 36.950654941997534]]], None, False)
collection = (ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date('2016-01-01', '2016-01-02'))
                                                .first())
cropLandcover = collection.select('cropland')
cropLandcover = cropLandcover.clip(AOI)
task = ee_imageToTask(cropLandcover, 30, crs='EPSG:32611')
task.start()



AOI = ee.Geometry.Point(-120.04305262182791,36.47617892554505)
collection = (ee.ImageCollection('EO1/HYPERION').filter(ee.Filter.date('2016-02-22', '2016-02-23')) # Target Path 42, Target Row 35
                                                .filterBounds(AOI)
                                                .first())


bands=['B008', 'B009', 'B010', 'B011', 'B012', 'B013', 'B014', 'B015', 'B016', 'B017',
        'B018', 'B019', 'B020', 'B021', 'B022', 'B023', 'B024', 'B025', 'B026', 'B027',
        'B028', 'B029', 'B030', 'B031', 'B032', 'B033', 'B034', 'B035', 'B036', 'B037',
        'B038', 'B039', 'B040', 'B041', 'B042', 'B043', 'B044', 'B045', 'B046', 'B047',
        'B048', 'B049', 'B050', 'B051', 'B052', 'B053', 'B054', 'B055', 'B056', 'B057',
        
        'B079', 'B080', 'B081', 'B082', 'B083', 'B084', 'B085', 'B086', 'B087', 
        'B088', 'B089', 'B090', 'B091', 'B092', 'B093', 'B094', 'B095', 'B096', 'B097', 
        'B098', 'B099', 'B100', 'B101', 'B102', 'B103', 'B104', 'B105', 'B106', 'B107', 
        'B108', 'B109', 'B110', 'B111', 'B112', 'B113', 'B114', 'B115', 'B116', 'B117', 
        'B118', 'B119', 'B120',
        
        'B131', 'B132', 'B133', 'B134', 'B135', 'B136', 'B137', 'B138', 'B139', 'B140', 
        'B141', 'B142', 'B143', 'B144', 'B145', 'B146', 'B147', 'B148', 'B149', 'B150', 
        'B151', 'B152', 'B153', 'B154', 'B155', 'B156', 'B157', 'B158', 'B159', 'B160', 
        'B161', 'B162', 'B163', 'B164', 'B165',
        
        'B181', 'B182', 'B183', 'B184', 'B185', 'B186', 'B187', 'B188', 'B189', 'B190', 
        'B191', 'B192', 'B193', 'B194', 'B195', 'B196', 'B197', 'B198', 'B199', 'B200', 
        'B201', 'B202', 'B203', 'B204', 'B205', 'B206', 'B207', 'B208', 'B209', 'B210', 
        'B211', 'B212', 'B213', 'B214', 'B215', 'B216', 'B217', 'B218', 'B219', 'B220', 
        'B221', 'B222', 'B223']

tasks = []
for band in bands:
    cropLandcover = collection.select([band])
    id = cropLandcover.id().getInfo()[:-2] + band
    tasks.append(ee_imageToTask(cropLandcover, 30, crs='EPSG:32611', id=id))

In [ ]:
for task in tasks:
    task.start()

## Downloads Landsat data

In [ ]:
folder = 'GEECollection'

#Function to mask clouds based on the pixel_qa band of Landsat SR data.
#param {ee.Image} image Input Landsat SR image
#return {ee.Image} Cloudmasked Landsat image
def cloudMaskL457(image):
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    # mask2 removes edge pixels that don't occur in all bands
    qa = image.select('pixel_qa')
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

def gapfill(image):
    return image.focal_mean()


coll_notmasked = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filter(ee.Filter.eq('WRS_PATH', 144))
                                                    .filter(ee.Filter.eq('WRS_ROW', 52))
                                                    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']))

coll_masked = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filter(ee.Filter.eq('WRS_PATH', 144))
                                                    .filter(ee.Filter.eq('WRS_ROW', 52))
                                                    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
                                                    .map(cloudMaskL457))


datesOI = ['2017-02-11', '2016-11-07', '2016-10-06', '2016-02-25', '2016-01-24', '2015-11-21', '2015-08-17', '2015-04-11', '2015-02-22', '2015-02-06',
           '2014-10-17', '2014-10-01', '2014-08-30', '2014-06-27', '2014-03-07', '2014-02-19', '2013-10-30', '2013-09-28', '2012-11-12', '2012-10-27']

def ee_dates_of_interest_from_collection(coll, datesOI):
    nImages = coll.size().getInfo()
    collList = coll.toList(nImages)
    collID = coll.getInfo()['id'].replace('/','_')
    tasks = []
    for i in range(nImages):
        img = ee.Image(collList.get(i))
        date =  datetime.datetime.fromtimestamp((img.date().getInfo()['value']) // 1000).strftime('%Y-%m-%d')
        if date in datesOI:
            tasks.append(ee_imageToTask(img, 30, collID))
    return tasks
def ee_execute_tasks(tasks):
    for task in tasks:
        task.start()

def ee_check_tasks_status(tasks):
    for task in tasks:
        print(ee.data.getTaskStatus(task.id)) # To check task state


tasks = ee_dates_of_interest_from_collection(coll_masked, datesOI)
ee_execute_tasks(tasks)


In [ ]:
ee_check_tasks_status(tasks)

In [ ]:
x = gdal.Open(os.getcwd() + '/drive/MyDrive/GEECollection/LE07_144052_20190116.tif', GA_Update)
x_arr = x.ReadAsArray()
print(x_arr.shape)

for i in range(1, 11+1):
    print(x.GetRasterBand(i).GetDescription())